# 금연구역_2차_주소전처리 시도

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# import tensorflow as tf

import requests
import json


In [10]:
PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
# PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

df_fail = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_실패(12833).csv', encoding ='cp949')
df_save = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_성공(32766).csv', encoding ='cp949')

## 실패한 주소 체크

전처리를 시도한 kakao 주소가 아닌 기존의 원주소로 다시 api 호출을 시도해봄

In [4]:
df_fail.shape, df_save.shape

((12833, 10), (32766, 10))

In [9]:
df_fail.head()

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,대전 보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,대전 태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,대전 선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN


`중구 대전 태평동 ` 같이 구 > 시 > 동 순으로 나열된 주소지들 존재함  
시 정보를 삭제

### 주소에서 대전 삭제

In [8]:
df_fail[df_fail['주소'].str.contains('대전')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
2,중구,어린이운송용 승합자동차,문화어린이집,대전 보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,대전 태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,대전 선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
10,중구,어린이운송용 승합자동차,꿈내리유치원,대전 산성동6-43,"대전 중구 산성동6-43(우성아파트 내), 꿈내리유치원",꿈내리유치원,중구 산성동6-43,NaN,NaN,NaN
11,중구,어린이운송용 승합자동차,청명초등사관학원,대전 보문산로257 1,"대전 중구 보문산로257 1,2층(문화동) 청명초등사관학원",2층 청명초등사관학원,중구 보문산로257 1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11519,유성구,어린이놀이시설,호산어린이공원,대전 학하동 730,대전 유성구 학하동 730,NaN,유성구 학하동 730,NaN,NaN,NaN
11520,유성구,어린이놀이시설,별빛어린이공원,대전 학하동 715-1,대전 유성구 학하동 715-1,NaN,유성구 학하동 715-1,NaN,NaN,NaN
11521,유성구,어린이놀이시설,학무정어린이공원,대전 학하동 763-4,대전 유성구 학하동 763-4,NaN,유성구 학하동 763-4,NaN,NaN,NaN
11522,유성구,어린이놀이시설,영당말어린이공원,대전 학하동 778-1,대전 유성구 학하동 778-1,NaN,유성구 학하동 778-1,NaN,NaN,NaN


In [11]:
# 주소에서 대전 삭제
df_fail["주소"] = df_fail["주소"].str.replace(pat='대전', repl=r' ', regex=True)
df_fail[df_fail['주소'].str.contains('대전')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도


#### 대전 제거 후 중복된 공백제거

In [13]:
print(df_fail['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_fail['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_fail['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df_fail['주소'].str.contains('     ').sum()) # 스페이스바 다섯번
print(df_fail['주소'].str.contains('      ').sum()) # 스페이스바 섯번

315
24
2
0
0


In [15]:
# 공백 제거
df_fail["주소"] = df_fail["주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_fail['주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_fail['주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_fail['주소'].str.contains('    ').sum()) # 스페이스바 네번
print(df_fail['주소'].str.contains('     ').sum()) # 스페이스바 다섯번
print(df_fail['주소'].str.contains('      ').sum()) # 스페이스바 섯번

0
0
0
0
0


In [16]:
df_fail.reset_index(drop=True, inplace=True)
df_fail.head()

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 대전 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 대전 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 대전 선화로57번길 8,NaN,NaN,NaN


In [18]:
df_road = df_fail.copy()

In [19]:
df_road['kakao 주소'] = df_road['구'].str.cat(df_road['주소'], sep=' ')

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12828,유성구,복합용도,식당,학하동676-1,대전광역시 유성구 학하동676-1,NaN,유성구 학하동676-1,NaN,NaN,NaN
12829,유성구,복합용도,NaN,학하동713-12,대전광역시 유성구 학하동713-12,NaN,유성구 학하동713-12,NaN,NaN,NaN
12830,유성구,복합용도,NaN,학하동716-7,대전광역시 유성구 학하동716-7,NaN,유성구 학하동716-7,NaN,NaN,NaN
12831,유성구,복합용도,NaN,화암동157-6,대전광역시 유성구 화암동157-6,NaN,유성구 화암동157-6,NaN,NaN,NaN


### kakao api 세팅

In [21]:
# # rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "616734aa8214dbf958cfaebd6bd41aec"

In [22]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [23]:
kakao = KakaoLocalAPI(rest_api_key)

In [24]:
add_list = df_road['kakao 주소']
add_list[0]

'중구 목중로26번길 '

In [25]:
kakao.search_address(add_list[0])

{'documents': [{'address': None,
   'address_name': '대전 중구 목중로26번길',
   'address_type': 'ROAD',
   'road_address': {'address_name': '대전 중구 목중로26번길',
    'building_name': '',
    'main_building_no': '',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.412961781981',
    'y': '36.3375130575055',
    'zone_no': ''},
   'x': '127.412961781981',
   'y': '36.3375130575055'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [26]:
# 2차 api 호출
for i in tqdm(range(len(add_list))):
    try :
        df_road['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_road['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_road['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_road.isnull().sum())

  1%|          | 77/12833 [00:04<11:08, 19.09it/s]C:\Users\JBY\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 12833/12833 [22:32<00:00,  9.49it/s] 

구               0
구분              0
시설명           659
주소              0
번지주소            0
세부주소        12328
kakao 주소        0
동            6737
위도           6737
경도           6737
dtype: int64


2차 api 호출 후 6096개 성공, 6737개 실패

### 2차 api 호출 성공/실패 데이터프레임 저장

In [27]:
df_fail2 = df_road[df_road['위도'].isnull()]
df_save2 = df_road[~df_road['위도'].isnull()]

print(df_fail2.shape)
print(df_save2.shape)

(6737, 10)
(6096, 10)


In [40]:
# 병합
df_save3= pd.concat([df_save2, df_save], 
                    axis=0,
                    join='outer'
                    )


df_save3.reset_index(drop=True)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,동구,사회복지시설,은어송마을1단지(아)경로당,청로 67,"대전 동구 동구청로 67, 은어송마을1단지(가오동)",은어송마을1단지,동구 청로 67,가오동,36.3056962273781,127.457215836363
1,동구,사회복지시설,미리내(아)경로당,청로203번길 35,"대전 동구 동구청로203번길 35,미리내@(판암동)",미리내아파트,동구 청로203번길 35,판암동,36.3159127178655,127.455110655522
2,동구,사회복지시설,은어송마을2단지(아)경로당,청로 35,"대전 동구 동구청로 35, 은어송마을2단지@(대성동)",은어송마을2단지아파트,동구 청로 35,대성동,36.3037469941274,127.458270508282
3,동구,사회복지시설,해드림실버요양원,청로 95,"대전 동구 동구청로 95, 8층 (가오동)",8층,동구 청로 95,가오동,36.3065069481971,127.456739020314
4,동구,어린이놀이시설,미리내@관리사무소,청로203번길 35,대전 동구 동구청로203번길 35(판암동),NaN,동구 청로203번길 35,판암동,36.3159127178655,127.455110655522
...,...,...,...,...,...,...,...,...,...,...
38857,동구,음식점,에꿍이치킨,충무로 244,대전광역시 동구 충무로 244 (신흥동),NaN,동구 충무로 244,신흥동,36.322869,127.442877
38858,동구,음식점,종가소머리곰탕,우암로 241,대전광역시 동구 우암로 241 (가양동),NaN,동구 우암로 241,가양동,36.344371,127.44301
38859,동구,음식점,다해어죽 가양점,가양로 142-3,대전광역시 동구 가양로 142-3 (가양동),NaN,동구 가양로 142-3,가양동,36.349017,127.441479
38860,동구,음식점,띵호와식당,동서대로 1702,대전광역시 동구 동서대로 1702 (가양동),NaN,동구 동서대로 1702,가양동,36.349428,127.438749


## 2차에서 실패한 6737개 중 번지주소로 호출 시도

In [34]:
df_fail3 = df_fail2.copy()
df_fail3

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12659,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN
12663,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,유성구 장대동225-35,NaN,NaN,NaN
12698,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4,NaN,NaN,NaN
12710,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,유성구 죽동150-11,NaN,NaN,NaN


In [35]:
road_list = df_fail3['번지주소']
road_list[0]

'대전광역시 중구 목중로26번길  , 83 (중촌동, 현대아파트)'

In [36]:
kakao.search_address(road_list[0])

{'documents': [{'address': {'address_name': '대전 중구 중촌동 116-8',
    'b_code': '3014010400',
    'h_code': '3014056000',
    'main_address_no': '116',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '중촌동',
    'region_3depth_name': '중촌동',
    'sub_address_no': '8',
    'x': '127.414436051345',
    'y': '36.3387070586325'},
   'address_name': '대전 중구 목중로26번길 83',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 중구 목중로26번길 83',
    'building_name': '',
    'main_building_no': '83',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.414436051345',
    'y': '36.3387070586325',
    'zone_no': '34804'},
   'x': '127.414436051345',
   'y': '36.3387070586325'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [37]:
# 3차 api 호출
for i in tqdm(range(len(road_list))):
    try :
        df_fail3['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_fail3['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_fail3['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_fail3.isnull().sum())

100%|██████████| 6737/6737 [09:23<00:00, 11.95it/s]

구              0
구분             0
시설명           29
주소             0
번지주소           0
세부주소        6236
kakao 주소       0
동           4796
위도          4796
경도          4796
dtype: int64


번지주소로 1941개 성공, 4796개 실패

In [38]:
df_fail_3rd = df_fail3[df_fail3['위도'].isnull()]
df_save_3rd = df_fail3[~df_fail3['위도'].isnull()]

print(df_fail_3rd.shape)
print(df_save_3rd.shape)

(4796, 10)
(1941, 10)


In [41]:
# 병합
df_save4= pd.concat([df_save_3rd, df_save3], 
                    axis=0,
                    join='outer'
                    )


df_save4.reset_index(drop=True)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,동구,게임제공업소,88오락실,고니길 4,"대전광역시 동구 고니길 4, 1층 (소제동)",1층,동구 고니길 4,가오동,36.3056962273781,127.457215836363
1,동구,게임제공업소,대전게임랜드,동서대로1683번길 46-7,"대전광역시 동구 동서대로1683번길 46-7, 1층 (용전동)",1층,동구 동서대로1683번길 46-7,판암동,36.3159127178655,127.455110655522
2,동구,음식점,서울분식식당,천동초등동길 53,"대전광역시 동구 천동초등동길 53, 1층 (천동)",1층,동구 천동초등동길 53,대성동,36.3037469941274,127.458270508282
3,동구,음식점,제주산 해뜨는바다,동서대로 1682-1,"대전광역시 동구 동서대로 1682-1, 1층(성남동)",1층,동구 동서대로 1682-1,가오동,36.3065069481971,127.456739020314
4,동구,어린이운송용 승합자동차,키즈드림어린이집,계족로140번길 33 104-104,"대전광역시 동구 계족로140번길 33 104-104 (대동,펜타뷰)",NaN,동구 계족로140번길 33 104-104,판암동,36.3159127178655,127.455110655522
...,...,...,...,...,...,...,...,...,...,...
40798,동구,음식점,에꿍이치킨,충무로 244,대전광역시 동구 충무로 244 (신흥동),NaN,동구 충무로 244,신흥동,36.322869,127.442877
40799,동구,음식점,종가소머리곰탕,우암로 241,대전광역시 동구 우암로 241 (가양동),NaN,동구 우암로 241,가양동,36.344371,127.44301
40800,동구,음식점,다해어죽 가양점,가양로 142-3,대전광역시 동구 가양로 142-3 (가양동),NaN,동구 가양로 142-3,가양동,36.349017,127.441479
40801,동구,음식점,띵호와식당,동서대로 1702,대전광역시 동구 동서대로 1702 (가양동),NaN,동구 동서대로 1702,가양동,36.349428,127.438749


### 2차 api호출 시도를 csv 파일로 저장


In [43]:
df_save4.shape, df_fail_3rd.shape

((40803, 10), (4796, 10))

In [44]:
df_save4.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(40803).csv", index=False, encoding='cp949')
df_fail_3rd.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_실패(4796).csv", index=False, encoding='cp949')